# Fake News And Post Detector (Upload this notebook to colab)

In [2]:
# Install required packages
!pip install torch transformers datasets scikit-learn pandas tensorflowjs tqdm flask

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 






**Import dataset from hugging face to train**

In [3]:
from huggingface_hub import login
from google.colab import userdata
login(userdata.get('HF_TOKEN'))  #Hugging Face Api key is stored in colab secrets
from datasets import load_dataset
dataset = load_dataset('liar')
print(dataset['train'][0])  # Check a sample. Press y when prompted

README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

liar.py:   0%|          | 0.00/6.41k [00:00<?, ?B/s]

The repository for liar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/liar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1283 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1284 [00:00<?, ? examples/s]

{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer'}


**preprocess**

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess(data):
    labels = [0 if x <= 1 else 1 for x in data['label']]  # 0-1 = fake, 2-5 = credible
    encodings = tokenizer(data['statement'], truncation=True, padding=True, max_length=128)
    return {'input_ids': encodings['input_ids'], 'attention_mask': encodings['attention_mask'], 'labels': labels}

train_data = dataset['train'].map(preprocess, batched=True)
test_data = dataset['test'].map(preprocess, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/10269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1283 [00:00<?, ? examples/s]

In [5]:
!pip install wandb

For Visualization

In [6]:
import wandb
wandb.login()  # This opens a prompt in Colab

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tanmay2004desai (tanmay2004desai-padre-conceicao-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

**Train Model**

In [8]:
from datasets import load_dataset
dataset = load_dataset('liar', trust_remote_code=True)
print(dataset['train'][0])

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocess with consistent padding
def preprocess(data):
    labels = [0 if x <= 1 else 1 for x in data['label']]  # 0-1 = fake, 2-5 = credible
    encodings = tokenizer(data['statement'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')
    return {
        'input_ids': encodings['input_ids'].squeeze(),  # Remove extra batch dim
        'attention_mask': encodings['attention_mask'].squeeze(),
        'labels': labels
    }

# Apply preprocessing
train_data = dataset['train'].map(preprocess, batched=True)
test_data = dataset['test'].map(preprocess, batched=True)

# Set format for PyTorch
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  #tried with 10 epochs but it overfitted for unknown reason
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Updated from evaluation_strategy
    report_to="none"  # Disable W&B for simplicity (optional)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
)

trainer.train()

{'id': '2635.json', 'label': 0, 'statement': 'Says the Annies List political group supports third-trimester abortions on demand.', 'subject': 'abortion', 'speaker': 'dwayne-bohac', 'job_title': 'State representative', 'state_info': 'Texas', 'party_affiliation': 'republican', 'barely_true_counts': 0.0, 'false_counts': 1.0, 'half_true_counts': 0.0, 'mostly_true_counts': 0.0, 'pants_on_fire_counts': 0.0, 'context': 'a mailer'}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.640900,0.664956
2,0.662900,0.665920
3,0.487500,0.823761


TrainOutput(global_step=3852, training_loss=0.6271310368492844, metrics={'train_runtime': 943.0978, 'train_samples_per_second': 32.666, 'train_steps_per_second': 4.084, 'total_flos': 2026415570618880.0, 'train_loss': 0.6271310368492844, 'epoch': 3.0})

Test

In [9]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)
print(classifier("Aliens built the pyramids!"))  #Test the model

Device set to use cuda:0


[{'label': 'LABEL_0', 'score': 0.8973508477210999}]


**Store The Model In Drive**

In [10]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained('/content/drive/MyDrive/fake_news_model')
tokenizer.save_pretrained('/content/drive/MyDrive/fake_news_model')

Mounted at /content/drive


('/content/drive/MyDrive/fake_news_model/tokenizer_config.json',
 '/content/drive/MyDrive/fake_news_model/special_tokens_map.json',
 '/content/drive/MyDrive/fake_news_model/vocab.txt',
 '/content/drive/MyDrive/fake_news_model/added_tokens.json')

**Model Test**

In [11]:
from transformers import pipeline

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

# Test samples
samples = [
    "Aliens built the pyramids!",
    "The Earth is flat and NASA lies!",
    "Obama was born in Kenya.",
    "It rained in New York yesterday.",
    "Says the Annies List political group supports third-trimester abortions on demand."
]

for text in samples:
    result = classifier(text)
    label = 'Fake' if result[0]['label'] == 'LABEL_0' else 'Credible'
    print(f"{text}: {label} ({result[0]['score']:.2f})")

Device set to use cuda:0


Aliens built the pyramids!: Fake (0.90)
The Earth is flat and NASA lies!: Credible (0.58)
Obama was born in Kenya.: Credible (0.62)
It rained in New York yesterday.: Credible (0.94)
Says the Annies List political group supports third-trimester abortions on demand.: Fake (0.90)


**Run Model On Server To Send Request From Anywhere (Browser Extension)**

In [2]:
!pip install flask_cors pyngrok

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from flask import Flask, request, jsonify
from transformers import pipeline
from pyngrok import ngrok
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all origins
classifier = pipeline('text-classification', model='/content/drive/MyDrive/fake_news_model')

@app.route('/analyze', methods=['POST'])
def analyze():
    text = request.json['text']
    result = classifier(text)
    label = 'Fake' if result[0]['label'] == 'LABEL_0' else 'Credible'
    return jsonify({'label': label, 'score': result[0]['score']})

ngrok.set_auth_token("2srC7TM9BMZPITbWEzvfW2GcPGo_4ifufznpQGdjhGxGJUXon")  #go to ngrok website to get auth
#copy the public url which is the Flask Api and paste in  vscode
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
app.run(host='0.0.0.0', port=5000)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Device set to use cpu


Public URL: NgrokTunnel: "https://2dd4-34-19-120-216.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [10/Mar/2025 16:31:09] "OPTIONS /analyze HTTP/1.1" 200 -
Token indices sequence length is longer than the specified maximum sequence length for this model (1333 > 512). Running this sequence through the model will result in indexing errors
ERROR:__main__:Exception on /analyze [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/